# 서울 경찰청 검색

## 데이터 가져오기

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import math## 데이터 가져오기

In [2]:
# 드라이버로 링크 열기
url = 'https://www.police.go.kr/user/bbs/BD_selectBbsList.do?q_bbsCode=1038&q_tab=1'
driver = webdriver.Chrome('chromedriver')
driver.get(url)
time.sleep(5)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [9]:
station_list = soup.find('tbody').find_all('tr')

In [10]:
station = station_list[0]

In [13]:
# 관서명, 주소, 홈페이지
name = station.select('.show-col')[0].get_text().strip()
addr = station.select('.show-col')[1].get_text().strip()
website = station.select('.show-col')[2].get_text().strip()

'서울특별시경찰청'

In [16]:
lis = []
for station in station_list:
    name = station.select('.show-col')[0].get_text().strip()
    addr = station.select('.show-col')[1].get_text().strip()
    website = station.select('.show-col')[2].get_text().strip()
    lis.append([name, addr, website])

In [18]:
df = pd.DataFrame(lis, columns = ['관서명', '주소', '홈페이지'])
df

,관서명,주소,홈페이지
0,서울특별시경찰청,서울시 종로구 사직로8길 31,https://www.smpa.go.kr/
1,서울강남경찰서,서울시 강남구 테헤란로 114길 11,https://www.smpa.go.kr/gn/
2,서울강동경찰서,서울시 강동구 성내로 57,https://www.smpa.go.kr/gd/
3,서울강북경찰서,서울시 강북구 오패산로 406,https://www.smpa.go.kr/gb/
4,서울강서경찰서,서울시 강서구 화곡로 308,https://www.smpa.go.kr/gs/
5,서울관악경찰서,서울시 관악구 관악로5길 33,https://www.smpa.go.kr/ga/
6,서울광진경찰서,서울시 광진구 자양로 167,https://www.smpa.go.kr/gj/
7,서울구로경찰서,서울시 구로구 가마산로 235,https://www.smpa.go.kr/gr/
8,서울금천경찰서,서울시 금천구 시흥대로73길 50,https://www.smpa.go.kr/gc/
9,서울남대문경찰서,서울시 중구 한강대로 410,https://www.smpa.go.kr/ndm/


In [31]:
df['주소'].loc[28] = '서울시 성북구 화랑로7길 32'

In [29]:
df.to_csv('서울경찰서.csv')

## 지도에 표시하기

In [20]:
import folium
import json
from urllib.parse import quote

In [21]:
# api 키 불러오기
with open('카톡_api_key.txt') as f:
    api_key = f.read()

In [22]:
search_url = "https://dapi.kakao.com/v2/local/search/address.json"

In [36]:
# 지도
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

# 지도에 소방기관 표시
for i in df.index:
    try:
        road_addr = df['주소'].loc[i]
        station_name = df['관서명'].loc[i]
        station_website = df['홈페이지'].loc[i]
        url = f'{search_url}?query={quote(road_addr)}'
        result = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}).json()
        lng = float(result['documents'][0]['x'])
        lat = float(result['documents'][0]['y'])
        folium.Marker(
            location = [lat, lng], 
            popup = folium.Popup(road_addr+"\n"+station_website, max_width=200),
            tooltip = station_name, 
            icon = folium.Icon(color = 'darkblue', icon = 'glyphicon glyphicon-home')
        ).add_to(map)
    except:
        print(station_name)
title = "<h3 align = 'center' style = 'font-size:20px'> 서울소재 경찰서 위치</h3>"
map.get_root().html.add_child(folium.Element(title))
map